In [1]:
import torch
from torchdrug import datasets, tasks, core, models

dataset = datasets.ZINC250k("~/molecule-datasets/", kekulize=True,
                           atom_feature="symbol")

Loading /Users/ca/molecule-datasets/250k_rndm_zinc_drugs_clean_3.csv:  50%|█████     | 249456/498911 [00:03<00:03, 65720.70it/s]
Constructing molecules from SMILES: 100%|██████████| 249455/249455 [06:32<00:00, 636.18it/s] 


In [ ]:
from torchdrug.layers import functional
graph = dataset[0]['graph']
node_feature = functional.one_hot(graph.atom_type, dataset.node_feature_dim)
edge_feature = functional.one_hot(graph.edge_list[:,-1], dataset.edge_feature_dim)
graph.edge_feature = edge_feature